# Using Type Hints (15 mins)

One of Fugue's core philosophies is adapting to data practitioners to let them define their logic in the most native grammar. In effect, this:

* reduces boilerplate code
* reduces framework lock-in
* increases maintainability

## Example Use Case

## Natural Language Processing



### Distributed API Calls

Below we show how to parallelize expensive API calls easily using Fugue.

In [ ]:
import pandas as pd
from typing import List
import time

In [4]:
import requests as re

res = re.get("https://pokeapi.co/api/v2/pokemon/ditto")
res.json()

In [29]:
pokemon_to_get = pd.DataFrame({"pokemon":["ditto", "pikachu", "bulbasaur", "squirtle", "geodude", "charmander", "articuno", "jigglypuff"]})
pokemon_to_get.head()

,pokemon
0,ditto
1,pikachu
2,bulbasaur
3,squirtle
4,geodude


In [21]:
base_url = "https://pokeapi.co/api/v2/pokemon/"

def expensive_api(url):
    time.sleep(3)
    return re.get(url).json()

In [16]:
def get_data(pokemon: pd.DataFrame) -> pd.DataFrame:
    names = []
    id_numbers = []
    types = []
    for item in pokemon.values.tolist():
        res = expensive_api(f"{base_url}{item[0]}")
        names.append(res['forms'][0]['name'])
        id_numbers.append(res['id'])
        types.append(res['types'][0]['type']['name'])
    return pd.DataFrame({'name': names, 'id': id_numbers, 'type': types})

call_api(list_of_pokemon)

,name,id,type
0,ditto,132,normal
1,pikachu,25,electric
2,bulbasaur,1,grass
3,squirtle,7,water
4,geodude,74,rock


In [26]:
from typing import Iterable, List, Any, Dict

def get_data_native(pokemon: List[List[Any]]) -> Iterable[Dict[str,Any]]:
    for item in pokemon:
        res = expensive_api(f"{base_url}{item[0]}")
        yield {"name": res['forms'][0]['name'],
               "id": res['id'],
               "type": res['types'][0]['type']['name']}

In [27]:
from fugue import transform
transform(pokemon_to_get, get_data_native, schema="name: str, id: int, type: str")

,name,id,type
0,ditto,132,normal
1,pikachu,25,electric
2,bulbasaur,1,grass
3,squirtle,7,water
4,geodude,74,rock


In [28]:
transform(pokemon_to_get, 
          get_data_native, 
          schema="name: str, id: int, type: str",
          engine="dask").compute()

,name,id,type
0,ditto,132,normal
0,pikachu,25,electric
0,bulbasaur,1,grass
0,squirtle,7,water
0,geodude,74,rock


### Exercise

Create a function to call the [Dinosaur API](https://dinosaur-facts-api.shultzlab.com/) random endpoint. It can take any input/output type that Fugue supports.

For input, the suggested ones are:

* `List[Dict[str,Any]]`
* `pd.DataFrame`
* `List[List[Any]]`

For output, the suggested ones are:

* `Iterable[Dict[str,Any]]`
* `pd.DataFrame`
* `List[List]`

Remember that you can mix and match.

In [31]:
df = pd.DataFrame({"item": [1,2,3,4,5,6,7,8]})
df.head()

,item
0,1
1,2
2,3
3,4
4,5


`transform(df, fn, schema="name:str, description:str")`

## Increased Maintainability